Рассмотрим работу библиотеки SB3 для игр от Atari.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gymnasium
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]
!pip install stable-baselines3[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=11660fb5d92e8c619afd58e577b2be5a032722d7531527f4e3c3539aab5d12d1
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidi

In [1]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

import ale_py

# регистрируем среды в gym
gym.register_envs(ale_py)

In [2]:
env = make_atari_env("ALE/Breakout-v5", n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)

log_path = os.path.join('logs', 'SB3_A2C_Breakout')
a2c_path = os.path.join('savedModels', 'SB3_A2C_Breakout')
save_path_best = os.path.join('savedModels', 'SB3_A2C_Breakout_best')
# save_path_best = '/content/drive/MyDrive/models/breakout_SB3_2M_best.pkl'

In [13]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

stop_callback = StopTrainingOnRewardThreshold(reward_threshold=400, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=20000,
                             best_model_save_path=save_path_best,
                             verbose=1)

In [15]:
#model = A2C("CnnPolicy", env, verbose=1, tensorboard_log=log_path)
model = A2C("CnnPolicy", env, tensorboard_log=log_path)

In [5]:
model = A2C.load('/content/drive/MyDrive/models/breakout_SB3_2M.pkl', env)

FileNotFoundError: [Errno 2] No such file or directory: '\\content\\drive\\MyDrive\\models\\breakout_SB3_2M.pkl.zip'

In [ ]:
model.learn(total_timesteps=10000000, callback=eval_callback, progress_bar=True)

Выходные данные были обрезаны до нескольких последних строк (5000).
|    value_loss         | 0.0726    |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 983      |
|    ep_rew_mean        | 30.4     |
| time/                 |          |
|    fps                | 202      |
|    iterations         | 20700    |
|    time_elapsed       | 2043     |
|    total_timesteps    | 414000   |
| train/                |          |
|    entropy_loss       | -0.0199  |
|    explained_variance | 0.931    |
|    learning_rate      | 0.0007   |
|    n_updates          | 177451   |
|    policy_loss        | -0.00196 |
|    value_loss         | 0.0441   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1e+03    |
|    ep_rew_mean        | 31.2     |
| time/                 |          |
|    fps                | 202      |
|    

In [11]:
model.save(a2c_path)

In [17]:
#model = A2C.load('/content/drive/MyDrive/models/breakout_SB3_2M.pkl', env)
model = A2C.load(save_path_best, env)

FileNotFoundError: [Errno 2] No such file or directory: 'savedModels\\SB3_A2C_Breakout_best.zip'

In [19]:
model.policy

ActorCriticCnnPolicy(
  (features_extractor): NatureCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=3136, out_features=512, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): NatureCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=3136, out_features=512, bias=True)
      (1): ReLU()
    )
  )
  (vf_features_extractor): NatureCNN(
    (cnn): 

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(21.9, 5.107837115648854)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=1, render=True)

In [21]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [23]:
environment_name = "CarRacing-v2"
env = gym.make(environment_name)

D:\anaconda3\Lib\site-packages\gymnasium\envs\registration.py:517: DeprecationWarning: WARN: The environment CarRacing-v2 is out of date. You should consider upgrading to version `v3`.
  logger.deprecation(


DeprecatedEnv: Environment version v2 for `CarRacing` is deprecated. Please use `CarRacing-v3` instead.

In [27]:
save_path = os.path.join('savedModels', 'SB3_PPO_CarRacing')
save_path_best = os.path.join('savedModels', 'SB3_PPO_CarRacing_best')
log_path = os.path.join('logs', 'SB3_PPO_CarRacing')

In [ ]:
#save_path_best = os.path.join('/content/drive/MyDrive/models/', 'SB3_PPO_CarRacing_best')

In [29]:
# создание callback
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=3000, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=25000,
                             best_model_save_path=save_path_best,
                             verbose=1)

In [31]:
env = gym.make('CarRacing-v2')

model = PPO("CnnPolicy", env, verbose=1)

DeprecatedEnv: Environment version v2 for `CarRacing` is deprecated. Please use `CarRacing-v3` instead.

In [39]:
env = gym.make('CarRacing-v2', render_mode = 'human')
evaluate_policy(model, env, n_eval_episodes=1, render=True)

DeprecatedEnv: Environment version v2 for `CarRacing` is deprecated. Please use `CarRacing-v3` instead.

In [13]:
model.learn(total_timesteps=400000, callback=eval_callback)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/callbacks.py:414: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x7a6934890130> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7a693ff31ed0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -54.1    |
| time/              |          |
|    fps             | 79       |
|    iterations      | 1        |
|    time_elapsed    | 25       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -51.9       |
| time/                   |             |
|    fps                  | 75          |
|    iterations           | 2           |
|    time_elapsed         | 54          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008198565 |
|    clip_fraction        | 0.0749      |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.22       |
|    explained_variance   | -0.000963   |
|    learning_rate        | 0.

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=25000, episode_reward=40.35 +/- 75.92
Episode length: 906.60 +/- 186.80
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 907         |
|    mean_reward          | 40.4        |
| time/                   |             |
|    total_timesteps      | 25000       |
| train/                  |             |
|    approx_kl            | 0.022186568 |
|    clip_fraction        | 0.218       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.99       |
|    explained_variance   | 0.75        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00366     |
|    n_updates            | 120         |
|    policy_gradient_loss | -0.0234     |
|    std                  | 0.911       |
|    value_loss           | 0.162       |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     

In [ ]:
'/content/drive/MyDrive/models'

In [14]:
model.save('/content/drive/MyDrive/models/car_500K.pkl')

In [ ]:
env = gym.make('CarRacing-v2', render_mode = 'human')
#model = PPO("CnnPolicy", env, verbose=1)
model = PPO.load('savedModels/car_500K.pkl', env=env)


In [8]:
env = gym.make('CarRacing-v3')
model = PPO("CnnPolicy", env, verbose=1)
model.load('savedModels/car_500K.pkl', env)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


D:\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
D:\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


In [ ]:
model2.load('/content/drive/MyDrive/models/car_100K.pkl', env)

In [41]:
env = gym.make('CarRacing-v3', render_mode = 'human')
model = PPO.load('savedModels/car_500K.pkl', env=env)
evaluate_policy(model, env, n_eval_episodes=1, render=True)

D:\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
D:\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


(892.5373338237405, 0.0)

In [14]:
env.close()